# Heuristic performance

In [1]:
import ast

import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import seaborn as sns
import pandas as pd
import numpy as np
from matplotlib import gridspec

In [2]:
plt.rcParams['lines.linewidth'] = 2
plt.rcParams['lines.markersize'] = 8
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['axes.titlesize'] = 16
plt.rcParams['legend.title_fontsize'] = 14
plt.rcParams['legend.fontsize'] = 12

In [3]:
pd.set_option('display.max_rows', None)  # display all rows.
pd.set_option('display.float_format', '{:.2f}'.format)  # two decimals.

plt.style.use('grayscale')

%matplotlib inline

In [4]:
def get_cached_data(method, num_exps):
    data = []

    for experiment in range(1, num_exps + 1):
        df = pd.read_csv(f"cache/{experiment}-{method}.csv")
        df["experiment"] = experiment

        data.append(df)

    return pd.concat(data, ignore_index=True).set_index(["experiment", "instance"])

In [24]:
experiments = pd.read_csv("experiments/experiments.csv", index_col="index").rename_axis(index="experiment")
experiments.head()

,learners,instances,penalty,min_batch,max_batch,progress,preferences,qualifications,split,courses,modules
experiment,,,,,,,,,,,
1,800,1,0.50,5,30,0,2,"(1, 0, 0)",True,"[(4, 1), (4, 1), (3, 1), (4, 1), (2, 1), (1, 2...",48
2,1200,1,0.50,5,30,0,2,"(1, 0, 0)",True,"[(4, 1), (4, 1), (3, 1), (4, 1), (2, 1), (1, 2...",48
3,1600,1,0.50,5,30,0,2,"(1, 0, 0)",True,"[(4, 1), (4, 1), (3, 1), (4, 1), (2, 1), (1, 2...",48
4,800,1,0.75,5,30,0,2,"(1, 0, 0)",True,"[(4, 1), (4, 1), (3, 1), (4, 1), (2, 1), (1, 2...",48
5,1200,1,0.75,5,30,0,2,"(1, 0, 0)",True,"[(4, 1), (4, 1), (3, 1), (4, 1), (2, 1), (1, 2...",48


In [10]:
ilp = get_cached_data("ilp", 144)
ilp.head()

objective                          bounds  iterations  \
experiment instance                                                          
1          1           4739.50            [4739.502, 4739.502]          56   
           2           4733.91   [4733.912, 4733.933545454547]          67   
           3           4773.89  [4773.8855, 4774.334687499997]          34   
           4           4876.25     [4876.25, 4876.40977777778]          39   
           5           4734.33          [4734.3285, 4734.3285]          42   

                     run-time (wall)  instruction (# learners)  \
experiment instance                                              
1          1                   28.14                       786   
           2                   30.76                       787   
           3                   25.82                       785   
           4                   39.45                       791   
           5                   24.99                       787   

                     self-study (# learners)  activities (#)  \
experiment instance                                            
1          1                              14              70   
           2                              13              66   
           3                              15              68   
           4                               9              65   
           5                              13              66   

                                            instruction activity sizes  \
experiment instance                                                      
1          1         [14, 12, 5, 11, 16, 7, 7, 11, 12, 13, 12, 5, 9...   
           2         [12, 13, 5, 12, 5, 16, 16, 14, 13, 11, 11, 16,...   
           3         [14, 15, 12, 10, 7, 14, 5, 14, 15, 5, 7, 11, 5...   
           4         [15, 13, 5, 10, 11, 16, 6, 7, 8, 14, 16, 9, 12...   
           5         [9, 14, 11, 10, 15, 10, 14, 13, 13, 12, 14, 11...   

                    self-study activity sizes  gap (10 min)  
experiment instance                                          
1          1                             [14]          0.00  
           2                             [13]          0.00  
           3                             [15]          0.01  
           4                              [9]          0.00  
           5                             [13]          0.00

In [8]:
# heuristic = get_cached_data("heuristic", 144)
# heuristic.head()

FileNotFoundError: [Errno 2] No such file or directory: 'cache/1-heuristic.csv'

## ILP gaps after 10 min

In [33]:
data = ilp.join(experiments)
data['gap (10 min)'].groupby([data.progress, data.learners]).mean()

progress  learners
0         800        0.00
          1200       0.01
          1600       0.01
1         800        0.08
          1200       0.29
          1600       0.19
2         800        0.27
          1200       0.97
          1600       2.10
3         800        0.34
          1200       1.46
          1600       3.32
Name: gap (10 min), dtype: float64

## Heuristic gaps